
# Test ROC curves
ROC curves hasn't always been explained that well, but a they are important and a lot of time is spent by the book using them as a mean of comparing classifiers.
In this notebook I'll try to clarify some concepts that has been illustrated during the course, with the hope of giving another point of view to better incorporate such concept. 